# <b> SYRIATEL CHURN ANALYSIS </b>

#### Author : Stella Kitur
--- 
## <b> Project Overview </b>
In this project I have utilised machine learning algorithms in order to identify any trends that can help in predicting whether a customer that is using SyriaTel will stop (churn) using the services. This is to help SyriaTel in their decision making process as well as in developing methods that might help to reduce the churn rate further.

## <b> Business Understanding </b>
### <b> Business Problem </b>
- Who is SyriaTel?
- Customer retention rate is key in the telecommunication domain 
- Important to know what features are leading to churn rate increase 
- What are the potential factors for churn?
SyriaTel is a Telecommunication company 
#### <b>Objectives</b>
As the data scientist assigned to this project, what are your objectives?
1. Identify if there are certain features that can predict whether a customer will churn or not
2. Predict as accurately as possible using a model, whether a customer will churn



#### <b> Metrics of Success </b>
In this model, the metrics of success are outlined as follows :


## <b> Data Understanding </b>
In conducting this analysis, the CRISP-DM data science process was used.
There were : Outline important notes based on the dataset... etc. etc.

---
# <b> Import Libraries </b>

To start off this analysis, we will import the libraries that will be used in this notebook.

As well as including the necessary formatting for the data visualisations used throughout the notebook.

For convenience, the libraries have been categorised based on the function. 

In [ ]:
# Import Libraries 

# Imports
import pandas as pd
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
custom_color = custom_colors = ["#BE5A83", "#F2B6A0", "#FEF2F4"] #This is the color pallette for the notebook


#Scikit imports
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import xgboost as xgb
from sklearn.feature_selection import RFECV
from sklearn.metrics import plot_confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


In [ ]:
# Load the data
# Display the shape of the data

df = pd.read_csv('data.csv')
print(df.shape)

## <b> Data Understanding </b>
Under this section, we will gain understanding of the dataset while also identifying if there are any missing/duplicated values before proceeding to conduct EDA (Exploratory Data Analysis) on the data to help us identify any key observations in the dataset. 

In [ ]:
# Display the first 5 rows of the dataset
df.head()

In [ ]:
# Display the information of the dataset
df.info()

### <b> Data Cleaning </b>

In [ ]:
# Check for missing values and duplicated values 
print(df.isnull().sum())
print(f"There are {df.duplicated().sum()} duplicated values.")


In [ ]:
# We will change the datatype of area code from an int to an object

df['area code'] = df['area code'].astype(object)
df['area code'].dtype # Check if the change has been made


In [ ]:
# Explore the values in the state column

print(df.state.value_counts())
print(f"There are {df['state'].nunique()} values, this is because SyriaTel is based in the USA and there are {df['state'].nunique()} states")


In [ ]:
# Drop the phone number column

df = df.drop('phone number',axis=1)
df.head() #inspect

In [ ]:
# Define a function that calls the categorical columns in the dataset
def print_categorical_columns(df):
    categorical_cols = df.select_dtypes(include='object').columns.difference(['phone number'])
    for col in categorical_cols:
        print(col.upper())
        print(df[col].unique())
        print('_________________________\n')

# Call the function
print_categorical_columns(df)

#### <b> Label Encoding and One-Hot Encoding </b>

In [ ]:
# Label Encoding the State column
# Label Encoding is preferred in this case as there are 51 unique values and will replace it with a unique integer.

Label_Encoder = LabelEncoder()
df['state'] = Label_Encoder.fit_transform(df['state'])
df['state']

In [ ]:
# Convert categorical variables to binary representation
df["international plan"] = df["international plan"].map({"no": 0, "yes": 1})
df["voice mail plan"] = df["voice mail plan"].map({"no": 0, "yes": 1})
df['churn'] = df['churn'].map({False: 0, True: 1})

df.head(6)

##### <b> Feature Engineering </b>

In [ ]:
# Feature Engineering -- Total Expenditure 
# This will calculate the total expenditure for each customer


df['total expenditure'] = df['total day charge'] \
                        + df['total eve charge'] \
                        + df['total night charge'] \
                        + df['total intl charge']

df.head()

### <b> Exploratory Data Analysis </b>

##### <b> Summary Statistics </b>

In [ ]:
# Descriptive Summary Statistics 

df.describe()

##### <b> Distribution of Features</b>

In [ ]:
# The Distribution of Features

df.drop(columns='churn').hist(figsize=(18, 15), color="#BE5A83");


We notice based on this output that the features have different scalings, and we especially take note that not all of them are <b> normally distributed </b>

In [ ]:
# Display the count of churned and non-churned counts in a bar chart 

churn_counts = df["churn"].value_counts()

# Plot the bar chart
sns.countplot(x="churn", data=df, palette=custom_colors)
plt.xlabel("Churn")
plt.ylabel("Count")
plt.title("Customer Churn Distribution")
plt.xticks([0,1], ["Not Churned", "Churned"])
plt.show()

In [ ]:
# This function will be used to find the percentage value in different columns
def calculate_percentage(column):
    percentages = column.value_counts(normalize=True) * 100
    return percentages

In [ ]:
churn_percentages = calculate_percentage(df["churn"])
print(churn_percentages)

We can take note that majority of the customers 85.5% had not churned (2850), while 14.5 % had churned(483). 

In [ ]:
# Count the number of churned and non-churned customers by international plan
churn_intl_plan = df.groupby(['churn', 'international plan']).size().unstack()
total_churn_itl = churn_intl_plan.sum(axis=1)  # Calculate the total count for each churn category
percentage_intl_plan = churn_intl_plan.div(total_churn_itl, axis=0) * 100  # Calculate the percentage
percentage_intl_plan

In [ ]:
# Display as a bar chart

# Plots a stacked bar chart to visualize the relationship
churn_intl_plan.plot(kind='bar', stacked=True, figsize=(8, 6), color=custom_colors)

plt.xlabel('Churn')
plt.ylabel('Count')
plt.title('Churn Distribution by International Plan')
plt.xticks(rotation=0)
plt.legend(title='International Plan')

plt.show()

Observations:

- Among customers who did not churn (churn=False), approximately 93.50% have "no" international plan, and 6.50% have "yes" international plan.
- Among customers who churned (churn=True), approximately 71.64% have "no" international plan, and 28.36% have "yes" international plan.

In [ ]:
# Count the number of churned and non-churned customers by voicemail plan
churn_voicemail = df.groupby(['churn', 'voice mail plan']).size().unstack()
total_churn_vm= churn_voicemail.sum(axis=1)  # Calculate the total count for each churn category
percentage_vm = churn_voicemail.div(total_churn_vm, axis=0) * 100  # Calculate the percentage
percentage_vm

In [ ]:
# Display as a bar chart

# Plot a stacked bar chart to visualize the relationship
churn_voicemail.plot(kind='bar', stacked=True, figsize=(8, 6), color=custom_colors)

plt.xlabel('Churn')
plt.ylabel('Count')
plt.title('Churn Distribution by Voicemail Plan')
plt.xticks(rotation=0)
plt.legend(title='Voicemail Plan')

plt.show()

#### <b> Observations: </b>

- Churned customers (True): 83.44% did not have a voice mail plan (no), while 16.56% had a voice mail plan (yes).

- Non-churned customers (False): 70.46% did not have a voice mail plan (no), and 29.54% had a voice mail plan (yes).

In [ ]:
churn_area_code = df.groupby('area code')['churn'].value_counts().unstack() / 100
churn_area_code

In [ ]:
churn_area_code.plot(kind='bar', stacked=True, figsize=(12, 6), color=custom_colors)
plt.xlabel('Area Code', fontsize=12)
plt.ylabel('Percentage', fontsize=12)
plt.title('Churn by Area Code (Percentage)', fontsize=14)
plt.legend(title='Churn', loc='upper right')
plt.xticks(rotation=0)
plt.show()

Observations:


1. In area code <u>408</u>, there are 716 customers who did not churn, while 122 customers churned. The churn rate for this area code is relatively lower compared to the non-churn rate.

2. Area code <u>415</u> has a higher number of non-churned customers, with 1419 customers, compared to 236 customers who churned. 

3. In area code <u>510</u>, there are 715 non-churned customers, while 125 customers churned. 

### <b> Multivariate Analysis</b>

In [ ]:
# Creates a Correlation Matrix & then displays it as a heatmap
corr_matrix = df.corr()
corr_matrix

In [ ]:
# Display as a heat map
#using a heatmap to show correlation
fig, ax = plt.subplots(figsize=(16,15))
mask = np.triu(np.ones_like(df.corr(), dtype=bool))
sns.heatmap(df.corr(), linewidths=0.5, mask=mask, square=True, ax=ax, annot=True, cmap="RdPu");

In [ ]:
def check_multicollinearity(df, threshold=0.95):
    corr_matrix = df.select_dtypes(include=np.number).corr().abs()
    correlated_pairs = set()
    for col in corr_matrix:
        correlated_cols = corr_matrix.index[corr_matrix[col] > threshold]
        correlated_pairs.update([(min(col, correlated_col), max(col, correlated_col)) for correlated_col in correlated_cols if col != correlated_col])
    for pair in correlated_pairs:
        print(f"{pair[0]} --- {pair[1]}")
    return set(df.columns) & set(col for pair in correlated_pairs for col in pair)

# Call the function to check multicollinearity
multicollinear_features = check_multicollinearity(df)


##### <b> <u> Observations: </u></b>

The following pairs of features exhibit high correlation above the threshold of 0.95:

<ul>
<li> <code>total day minutes</code> and <code>total day charge</code></li>
<li> <code>total eve minutes</code> and <code>total eve charge</code></li>
<li> <code>total night minutes</code> and <code>total night charge</code></li>
<li> <code>total intl minutes</code> and <code>total intl charge</code></li>
</ul>


We can therefore take note that: 

There is a strong positve correlation between : 

- total day minutes and total day charge. This suggests that as the number of minutes spent on day calls increases, the corresponding charge for those calls also increases.

- total eve minutes and total eve charge. This indicates that higher evening call durations are associated with higher charges for those calls.

- total intl minutes is highly correlated with total intl charge. This indicates that longer international call durations are associated with higher charges for those calls.


In order to deal with the multicollinearity in the features, one of the features from each pair will have to be dropped. 



In [ ]:
# Drop some columns in order to deal with multicollinearity

df = df.drop(columns=['total day minutes', 'total eve minutes', 'total night minutes', 'total intl minutes'])
df.columns

#### <b> Data Preparation for ML Purposes 

#### Setting the target </b>

In [ ]:
X= df.drop('churn', axis=1)
y = df.churn
X.head() #Inspect new df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=123)


In [ ]:
X_train.info()

In [ ]:
X_train = X_train.drop('area code', axis=1).select_dtypes(include=['int', 'float'])
X_test = X_test.drop('area code', axis=1).select_dtypes(include=['int', 'float'])


In [ ]:
X_train.info() # inspect the changes made in the cell above

In [ ]:
# Scale the data

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Create an instance of SMOTE
smote = SMOTE(random_state=123)

# Resample the training data using SMOTE
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train) 

# Print the class distribution of the synthetic samples
class_distribution = pd.Series(y_train_resampled).value_counts()
print("Synthetic Sample Class Distribution:")
print(class_distribution)

In [ ]:
# SMOTE not applied to test data
y_test.value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

y_train_resampled.value_counts().plot(kind='pie', autopct='%.2f', textprops={'fontsize': 20}, colors=custom_colors, ax=ax)
ax.set_ylabel('Churn', fontsize=16)
ax.set_title('Churn distribution in percentage', fontsize=20);

The distribution of the churn classes is now balanced. SMOTE was applied on the training sets only.

This ensured that an accurate gauge can be made on the model's performance by using a raw test sample that has not been oversampled or undersampled.

# <b> Modeling

The models that are used in this analysis include: </b>

1. Logistic Regression

2. Decision Tree

3. Random Forest

4. K Nearest Neighbors


## <b>Logistic Regression Model </b>

In [ ]:
# Pipeline
pipe_log = Pipeline(steps=[('scale', StandardScaler()), ('logreg', LogisticRegression(fit_intercept=False, solver='liblinear'))])
pipe_log.fit(X_train_resampled, y_train_resampled)


In [ ]:
def evaluate(model, X_test, y_test, cmap='RdPu'):
    y_train_preds = model.predict(X_train_resampled)
    y_test_preds = model.predict(X_test)
    
    print('Recall Score:')
    print('Train:', recall_score(y_train_resampled, y_train_preds))
    print('Test:', recall_score(y_test, y_test_preds))
    
    print('\nPrecision Score:')
    print('Train:', precision_score(y_train_resampled, y_train_preds))
    print('Test:', precision_score(y_test, y_test_preds))
    
    print('\nAccuracy Score:')
    print('Train:', accuracy_score(y_train_resampled, y_train_preds))
    print('Test:', accuracy_score(y_test, y_test_preds))

    print('\nF1 Score:')
    print('Train:', f1_score(y_train_resampled, y_train_preds))
    print('Test:', f1_score(y_test, y_test_preds))
    
    cm = confusion_matrix(y_test, y_test_preds, labels=model.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
    
    fig, ax = plt.subplots(figsize=(8, 8))
    disp.plot(ax=ax, cmap=cmap)
    ax.set_title('Confusion Matrix')
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')
    
    plt.show()

In [ ]:
evaluate(pipe_log, X_test, y_test)


#### <b> Observations </b>

<b>1. Recall Score: </b>

The model achieves a recall of approximately 75% on the training data and 74% on the test data.

This means that the model is successful in correctly identifying around 74-75% of the actual positive cases in both sets.

The relatively consistent performance suggests that the model generalizes well.

<b>2. Precision Score:</b>

 The precision score indicates the proportion of correctly predicted positive cases out of all predicted positive cases.

The model achieves a precision of approximately 70% on the training data, but it drops significantly to around 27% on the test data.

This discrepancy suggests that the model may be prone to a large number of false positives when applied to unseen data.

<b>
3. Accuracy Score:</b>

The model attains an accuracy of approximately 72% on the training data and 70% on the test data.

While the accuracy is relatively high, it is important to note that accuracy alone may not provide a comprehensive assessment of model performance, especially in imbalanced datasets.

Overall, the model's performance seems to indicate some potential issues with generalization, as evidenced by the lower precision and slightly lower accuracy on the test data. It is advisable to further investigate and potentially fine-tune the model to improve its performance on unseen data.






#  <b> Non Parametric Models </b>
---
## <b> Decision Tree Model </b>



In [ ]:
# Create a pipeline
pipe_dt = Pipeline(steps=[('scale', StandardScaler()), ('clf', DecisionTreeClassifier(criterion='entropy', random_state=42))])
pipe_dt.fit(X_train_resampled, y_train_resampled)


In [ ]:
evaluate(pipe_dt, X_test, y_test)

In [ ]:
def plot_feature_importances(pipe, figsize, custom_color):
    model = pipe.steps[1][1]
    plt.figure(figsize=figsize)
    bars = plt.barh(X_train_resampled.columns, model.feature_importances_, align='center')
    
    # Set custom color for the bars
    for bar in bars:
        bar.set_color(custom_color)
    
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature')
    plt.show()

custom_color = "#BE5A83"  # Custom color for the bars to match the theme


In [ ]:
plot_feature_importances(pipe_dt, (10, 8), custom_color)


#### <b> Observations </b>

From the above visualisation, we observe the following are the most <u> key features </u> in determining whether a customer will churn or not. 

- Total Expenditure 

- Customer Service Calls

- Total intl charge

- Total night charge 

- Voicemail plan



In [ ]:
## Feature Selection 

rfecv = RFECV(estimator=DecisionTreeClassifier(random_state=42), scoring='recall')
pipe_dt2 = Pipeline(steps=[('scale', StandardScaler()), ('Feature Selection', rfecv), ('clf', DecisionTreeClassifier(random_state=42))])
pipe_dt2.fit(X_train_resampled, y_train_resampled)

In [ ]:
print(f'The optimal number of features are {rfecv.n_features_}' )


In [ ]:
# Rank these features - with 1

rfecv_df = pd.DataFrame(rfecv.ranking_,index=X_train_resampled.columns,columns=['Rank']).sort_values(by='Rank',ascending=True)
rfecv_df[rfecv_df['Rank'] == 1]

In [ ]:
## We will drop the features that are not optimal in predicting churn rate

cols = rfecv_df[rfecv_df['Rank'] == 1].index
X_train_resampled = X_train_resampled[cols]
X_test = X_test[cols]
X_train_resampled.head(3) # Inspects the dataframe

#### <b>HyperParameter Tuning for Decision Tree</b>

In [ ]:
params_dt = {'clf__criterion': ['gini', 'entropy'],
             'clf__max_depth': range(14, 32, 2),
             'clf__min_samples_split' : range(2, 10, 2),
             'clf__min_samples_leaf': [2, 3, 5, 7, 10],
             'clf__max_features': [11, 13, 15]
}

gridsearch_dt = GridSearchCV(pipe_dt, params_dt, cv=4, scoring='recall')
gridsearch_dt.fit(X_train_resampled, y_train_resampled)

In [ ]:
# parameters that gave the best result
print(f'The optimal parameters in this model are: {gridsearch_dt.best_params_}')
print()
# Mean cross-validated score of the best_estimator
print(f'The validation recall: {gridsearch_dt.best_score_}')

In [ ]:
# Evaluate the model
evaluate(gridsearch_dt, X_test, y_test)


In [ ]:
plot_feature_importances(pipe_dt, (10, 8), custom_color)

## <b> Random Forest Model </b>

In [ ]:
# # create a pipeline for Random Forest Model
pipe_rf = Pipeline(steps=[('scale', StandardScaler()), ('rf', RandomForestClassifier(random_state=42))])
pipe_rf.fit(X_train_resampled, y_train_resampled)

In [ ]:
# evaluate model performance
evaluate(pipe_rf, X_test, y_test)

#### <b> Hyperparameter tuning on the Random Forest Model </b>

In [ ]:
# hyperparameter tuning using GridSearchCV for Random Forest
params_rf = {'rf__n_estimators': range(400, 800, 200),
             'rf__criterion': ['gini', 'entropy'],
             'rf__max_depth': range(14, 20, 2),
             'rf__min_samples_split': range(3, 4, 7),
             'rf__min_samples_leaf': [5, 7, 12]
             
}

gridsearch_rf = GridSearchCV(pipe_rf, params_rf, cv=4, scoring='recall')
gridsearch_rf.fit(X_train_resampled, y_train_resampled)

In [ ]:
# parameters that gave the best result
print(f'The optimal parameters in this model are: {gridsearch_rf.best_params_}')
print()
# Mean cross-validated score of the best_estimator
print(f'The validation recall: {gridsearch_rf.best_score_}')

In [ ]:
# evaluate the performance of the model
evaluate(gridsearch_rf, X_test, y_test)

### <b> Support Vector Machine </b>

In [ ]:
# Create a pipeline for SVM
pipe_svm = Pipeline(steps=[('scale', StandardScaler()), ('svm', SVC(random_state=42))])

# Fit the pipeline on the resampled training data
pipe_svm.fit(X_train_resampled, y_train_resampled)


In [ ]:
evaluate(pipe_svm, X_test, y_test)


In [ ]:
params_svm = {
    'svm__C': [0.1, 1, 10],
    'svm__kernel': ['linear', 'rbf'],
    'svm__gamma': ['scale', 'auto']
}

# Perform grid search
gridsearch_svm = GridSearchCV(pipe_svm, params_svm, cv=4, scoring='recall')
gridsearch_svm.fit(X_train_resampled, y_train_resampled)


In [ ]:

# Get the best parameters and best score
best_params = gridsearch_svm.best_params_
best_score = gridsearch_svm.best_score_

# Print the best parameters and best score
print("Best Parameters:", best_params)
print("Best Score:", best_score)

In [ ]:
evaluate(gridsearch_svm, X_test, y_test)

---
# <b> Model Evaluation </b>


Based on the <u> Recall Scores </u> which was the main evaluation metrics in this classification model,
the best model to predict whether a customer would churn or not
is the <b><u>Decision Tree Model. </u></b>

This is due to the fact that it showed the best performance compared to all the other models that were utilised in this model.

<ul> <li> The recall score for the model was : 86.4 % </ul> </li>

<ul> <li> The optimal parameters in this model are: {'clf__criterion': 'entropy', 'clf__max_depth': 24, 'clf__max_features': 15, 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 2} </ul> </li>

In [ ]:
# evaluation metrics of the best model based on the test data
print('DECISION TREE(TUNED) SCORES:')
print('Test Recall score: ', recall_score(y_test, gridsearch_dt.predict(X_test)))
print('Test Precision score: ', precision_score(y_test, gridsearch_dt.predict(X_test)))
print('Test Accuracy score: ', accuracy_score(y_test, gridsearch_dt.predict(X_test)))
print('F1 Score: ' , f1_score(y_test, gridsearch_dt.predict(X_test)))

In [ ]:
plot_feature_importances(gridsearch_dt.best_estimator_, (10, 8), custom_color)

----
# <b> Conclusion & Recommendations </b>

Based on the model results, as the Data Scientist assigned to this project, I would recommend the following.

1. As <b> total expenditure </b> is an influencing factor for whether or not a customer will churn; 

    It is important that SyriaTel reconsiders some of the costs, perhaps in a way that would be more accomodating to individuals that 
have a certain budget. 

2. Additionally, focus should be placed on the issues that are raised during the <b> customer service calls </b>, while also ensuring that those who are responding to the customers needs are adequately trained as well as adhering to good customer service norms, in order to ensure quality service is provided. 

3. Furthermore, SyriaTel should consider taking a customer-centered approach, for example having certain plans that can be modified to suit the needs of the diverse customer base, example: some customers may be more interested in the international plan compared to having a voice mail plan.